## Components in LlamaIndex

### Create a Query engine for RAG

#### Setting up the persona database 
i will be using personas from the https://huggingface.co/datasets/dvilasuero/finepersonas-v0.1-tiny. This dataset contains 5K personas that will be attending the party!

Let's load the dataset and store it as files in the data directory


In [7]:
from datasets import load_dataset
from pathlib import Path

dataset = load_dataset(path="dvilasuero/finepersonas-v0.1-tiny", split="train")

Path("data").mkdir(parents=True, exist_ok=True)
for i, persona in enumerate(dataset):
    with open(Path("data") / f"persona_{i}.txt", "w") as f:
        f.write(persona["persona"])

ModuleNotFoundError: No module named 'datasets'

In [8]:
pip install datasets==3.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 4.6 MB/s eta 0:00:00a 0:00:01
  Using cached pandas-2.2.3-cp310-cp310-macosx_10_9_x86_64.whl (12.6 MB)
  Using cached xxhash-3.5.0-cp310-cp310-macosx_10_9_x86_64.whl (31 kB)
  Using cached dill-0.3.8-py3-none-any.whl (116 kB)
  Using cached fsspec-2024.12.0-py3-none-any.whl (183 kB)
  Using cached multiprocess-0.70.16-py310-none-any.whl (134 kB)
  Using cached pyarrow-19.0.1.tar.gz (1.1 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
  error: subprocess-exited-with-error
  
  × Building wheel for pyarrow (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [878 lines of output]
      /private/var/folders/mk/4kptng6s0x7_vcc6jw3xmjhh0000gn/T/pip-build-env-5dpd4l_c/overlay/lib/python3.10/site-packages/setuptools/c